In [ ]:
# !/usr/local/bin/python
# -*- coding: UTF-8 -*-
'''backtest
start: 2016-11-01 00:00:00
end: 2020-11-04 05:20:00
period: 1h
basePeriod: 15m
exchanges: [{"eid":"Binance","currency":"BTC_USDT","stocks":0}]
'''
import numpy as np
import talib
import pandas as pd

exchange.SetContractType('BTC_USDT')
# ------------------------------ 策略部分开始 --------------------------
print(exchange.GetAccount())  # 调用一些接口，打印其返回值。
initAccount = exchange.GetAccount()
print(exchange.GetTicker())
exchange.SetMaxBarLen(2000)  # 设置K线数量，最大一次两千根
stop_loss = None
stop_win = None  # 这两个应该是全局变量
wait_for = False  # 空头的开仓等待期
amount = 5000    # 1000个合约
# exchange.SetMarginLevel(3)    # 加x3杠杆
def BollingerBandsW(record):
    bbw_list = []
    boll = TA.BOLL(record)
    upLine = boll[0][21:]
    midLine = boll[1][21:]
    downLine = boll[2][21:]
    # try:
    for i in range(0, len(upLine) - 1):
        bbw = (upLine[-i] - downLine[-i]) / midLine[-i]
        print(bbw)
        bbw_list.append(bbw)
    # except TypeError:
    #   pass
    return bbw_list

def VolatilityLevel(record):
    alias = ('VolatilityLevelIndex',)
    lines = ('bbw', 'VLI_fast', 'VLI_top', 'VLI_slow',)
    params = (
        ('period', 20),
        ('devfactor', 2.0),
        ('fast_period', 20),
        ('slow_period', 100),
    )

    bbw = BollingerBandsW(record)
    VLI_fast = TA.MA(bbw, 200)[201:]
    VLI_slow = TA.MA(bbw, 1000)[1001:]
    VLI_top = []
    std = talib.STDDEV(np.array(bbw), 1000)[1001:]
    for i in range(0, len(VLI_slow) - 1):
        VLI_top.append(VLI_slow[-i] + 2 * std[-i])

    return bbw, VLI_fast, VLI_top, VLI_slow


# def log(str_):
  #  print(str_)
def onTick(e):
    global stop_loss
    global stop_win
    global wait_for
    ticker = exchange.GetTicker()
    account = exchange.GetAccount()
    position = exchange.GetPosition()
    kliens = exchange.GetRecords(PERIOD_H1)  # 五分钟的K线
    if len(kliens) < 1500:
        #Log('total bars: ', len(kliens))
        return
    bbw = BollingerBandsW(kliens)
    # B = VolatilityLevel(kliens)
    boll = TA.BOLL(kliens, 20, 2)
    upLine = boll[0]
    midLine = boll[1]
    downLine = boll[2]
    dataclose = kliens.Close
    datavolume = kliens.Volume
    datalow = kliens.Low
    sma_veryfast = TA.MA(dataclose, 10)
    sma_mid = TA.MA(dataclose, 50)
    sma_slow = TA.MA(dataclose, 100)
    sma_veryslow = TA.MA(dataclose, 200)
    sma_fast = TA.MA(dataclose, 20)
    volatility_level = VolatilityLevel(kliens)  # bbw,VLI_fast,VLI_top,VLI_slow
    VLI_fast, VLI_top, VLI_slow = volatility_level[1], volatility_level[2], volatility_level[3]
    ma_vol_10 = TA.MA(datavolume, 10)
    ma_vol_50 = TA.MA(datavolume, 50)
    low_volatility_level = VLI_fast[-1] < VLI_slow[-1]  # 这个要做成计数的两个都是列表
    high_volatility_level = VLI_fast[-1] > VLI_slow[-1]
    extreme_volatility_level = bbw > VLI_top  # 这个还暂时不清楚
    low_of_last_candle = kliens[-1]["Low"]  # 最新的K线？的最低价
    vol_condition = (ma_vol_10[-1] > ma_vol_50[-1])
    crossdown_boll_top = dataclose[-1] < upLine[-1] and dataclose[-2] > upLine[-2]  # 代替了crossdown
    # crossup_boll_bot = CrossUp(self.dataclose, self.boll.bot)
    crossup_boll_bot = dataclose[-1] > upLine[-1] and dataclose[-2] < upLine[-2]  # 代替了crossup
    highest_high = TA.Highest(kliens, 20, "Close")

    if exchange.GetOrders() != []:  # 如果有委托
        return

    if position == []:
        ismid_ = sma_mid[-1] < sma_slow[-1] and sma_slow[-1] < sma_veryslow[-1]
        exchange.SetDirection("buy")
        if crossdown_boll_top and vol_condition and position == []:
            if dataclose[-1] > sma_fast[-1]:
                if bbw[-1] < VLI_top[-1]:
                    if low_volatility_level:
                        if sma_mid[-1] > sma_veryslow[-1]:
                            exchange.Buy(ticker.Sell, amount)  # 市价单买入一手
                            stop_loss = low_of_last_candle * 0.95
                            return
                    elif not ismid_:  # 可以再加一个high_vol的条件
                        exchange.Buy(ticker.Sell, amount)  # 这些条件可以注释掉
                        stop_loss = low_of_last_candle * 0.95
                        return
                elif sma_slow[-1] > sma_veryslow[-1]:
                    exchange.Buy(ticker.Sell, amount)
                    stop_loss = low_of_last_candle * 0.95
                    return
                #stop_loss = low_of_last_candle * 0.95

        # if crossup_boll_bot and vol_condition:
        # 上面尝试开仓多头
        exchange.SetDirection("sell")
        if crossup_boll_bot and bbw[-1] < VLI_slow[-1]:
            # wait for 应该怎么判断
            wait_for = True
            if sma_slow[-1] < sma_mid[-1] and sma_slow[-2] > sma_mid[-2]:
                exchange.Sell(ticker.Buy, amount)  # 市价单卖出一手
                wait_for = False
                return

        if wait_for and sma_slow[-1] < sma_mid[-1] and sma_slow[-2] > sma_mid[-2] and bbw[-1] < VLI_top[-1]:
            exchange.Sell(ticker.Buy, amount)  # 市价单卖出一手
            wait_for = False
            return 
        # 上面尝试开仓空头
        # 上面尝试开仓空头

    portfolio_value = initAccount.Balance #account["Stocks"]
    # average_price = initAccount
    if position != None:    # position != []
        average_price = position.price
        trade_profit = average_price/portfolio_value
    else:
        trade_profit = 0
    # if trade_profit > 0.03:
    # stop_win = 0.01
    if trade_profit > 0.20:
        stop_win = 0.15
    elif trade_profit > 0.25:
        stop_win = 0.20
    elif trade_profit > 0.30:
        stop_win = 0.25
    elif trade_profit > 0.35:
        stop_win = 0.30
    elif trade_profit > 0.40:
        stop_win = 0.35

    if position !=None and len(position) > 0 and position[0].Type == 0:  # 表示在多仓的时候
        close_long = False  # 一开始假设不平仓
        if trade_profit < 0:
            if stop_loss != None:
                if ticker.Buy < stop_loss:
                    close_long = True
        elif stop_win != None:
            if trade_profit <= stop_win:
                close_long = True

        if (crossup_boll_bot and vol_condition) or close_long:  # close 的第一个条件
            exchange.SetDirection("closebuy")
            exchange.Sell(ticker.Buy, position[0].Amount)  # 全平
            stop_loss = None 
            stop_win = None  # 置零
            return

    if position !=None and len(position) > 0 and position[0].Type == 1:  # 表示在空头仓时
        """
        0-10 ： 20HH
        10-15 : 10HH
        15-25 : 20HH
        """
        close_sell = False
        highest_high_20 = TA.Highest(kliens[-2:], 20, "Low")
        highest_high_5 = TA.Highest(kliens[-2:], 5, "Low")
        highest_high_10 = TA.Highest(kliens[-2:], 10, "Low")
        if trade_profit < 0:
            # 止损
            if ticker.Sell > highest_high_20:
                close_sell = True

        elif stop_win!= None:
            if (trade_profit < stop_win):
                # 止盈条件
                close_sell = True
            elif trade_profit < 0.1 and ticker.Sell < highest_high_20:
                close_sell = True
            elif trade_profit > 0.1 and trade_profit < 0.15 and ticker.Sell < highest_high_10:
                close_sell = True
            elif trade_profit > 0.15 and trade_profit < 0.25 and ticker.Sell < highest_high_5:
                close_sell = True

        if close_sell:
            exchange.SetDirection("closesell")
            exchange.Buy(ticker.Sell, position[0].Amount)  # 全平
            stop_loss = None 
            stop_win = None  # 置零
            return
    if position !=None and position != []:
        if stop_win!=None:
            if trade_profit < stop_win:
                exchange.SetDirection("closebuy")
                #log("止损")
                exchange.Sell(ticker.Buy, position[0].Amount)  # 多头止损
                stop_loss = None
                stop_win = None
                return
        if stop_loss!=None:
            if dataclose[-1] > stop_loss:  # 空头止损
                exchange.SetDirection("closesell")
                #log("止损")
                exchange.Buy(ticker.Sell, position[0].Amount)
                stop_loss = None
                stop_win = None
                return
"""
    if crossup_boll_bot and vol_condition:
        exchange.SetDirection("sell")
        exchange.Sell(ticker.Buy, 1)
        stop_loss = highest_high
"""

def main():
    while True:
        onTick(exchange)
        Sleep(1000)